## RNN Feature extraction from sequences with various types of input using TF-Fold

In [1]:
# boilerplate
import numpy as np
import tensorflow as tf

sess = tf.InteractiveSession()
import tensorflow_fold as td

In [2]:
RNN_FEATURE_SIZE = 4
A_SIZE = 3
B_SIZE = 5

input_sequence = [
    {'type': 'A', 'data': [1, 2, 3]},
    {'type': 'B', 'data': [5, 4, 3, 2, 1]},
    {'type': 'A', 'data': [3, 2, 1]},
]

In [3]:
input_sequence

[{'data': [1, 2, 3], 'type': 'A'},
 {'data': [5, 4, 3, 2, 1], 'type': 'B'},
 {'data': [3, 2, 1], 'type': 'A'}]

In [4]:
feature_from_A = td.Vector(A_SIZE) >> td.FC(RNN_FEATURE_SIZE)
feature_from_B = td.Vector(B_SIZE) >> td.FC(RNN_FEATURE_SIZE)

In [5]:
feature_from_A.eval(input_sequence[0]['data'])

array([ 0.23814273,  0.        ,  0.        ,  0.        ], dtype=float32)

In [6]:
#feature_from_A.eval(input_sequence[1]['data'])  # fails since it gets the wrong size of input

### Data dependent path through computational graph

In [7]:
feature = td.OneOf(
    key_fn=lambda x: x['type'],
    case_blocks={
        'A': td.GetItem('data') >> feature_from_A,
        'B': td.GetItem('data') >> feature_from_B
    }
)

In [8]:
[feature.eval(input_) for input_  in input_sequence]

[array([ 0.23814273,  0.        ,  0.        ,  0.        ], dtype=float32),
 array([ 1.76643121,  0.        ,  0.58830833,  0.        ], dtype=float32),
 array([ 2.72293854,  0.        ,  0.        ,  0.0422411 ], dtype=float32)]

### Handling of arbitrary length sequences

In [9]:
feature_sequence = td.Map(feature)

In [10]:
feature_sequence.eval(input_sequence)

[array([ 0.23814273,  0.        ,  0.        ,  0.        ], dtype=float32),
 array([ 1.76643121,  0.        ,  0.58830833,  0.        ], dtype=float32),
 array([ 2.72293854,  0.        ,  0.        ,  0.0422411 ], dtype=float32)]

## Feed Features to RNN...

In [11]:
lstm_cell = td.ScopedLayer(
    tf.contrib.rnn.BasicLSTMCell(num_units=16),
    'lstm_cell'
)
lstm_output = feature_sequence >> td.RNN(lstm_cell, name='lstm')

In [12]:
lstm_output.eval(input_sequence)
# Format:
# (
#    [state_0, ... state_T],
#    (cell_T, state_T)
# )

([array([-0.01530655, -0.01208504, -0.01433548, -0.00552036, -0.00320932,
         -0.0082042 , -0.00801775, -0.00635356,  0.01204611, -0.01278908,
          0.00692911,  0.00645362, -0.00973393,  0.00959304, -0.00497879,
          0.01523228], dtype=float32),
  array([-0.17736126, -0.12461913, -0.13674751, -0.02024776, -0.02674744,
         -0.03523384, -0.06648903, -0.0595339 ,  0.06722662, -0.08778674,
          0.08348105,  0.06476323, -0.09577341,  0.11191335, -0.04952551,
          0.11275347], dtype=float32),
  array([-0.32331112, -0.24803369, -0.22155015, -0.05607499, -0.06755898,
         -0.08475923, -0.13961571, -0.12053503,  0.10680307, -0.11922057,
          0.15261184,  0.14180364, -0.18945561,  0.2370128 , -0.10396434,
          0.21951388], dtype=float32)],
 (array([-0.59799623, -0.43577787, -0.58478981, -0.10034901, -0.10941851,
         -0.25983173, -0.21727414, -0.21551225,  0.24989481, -0.34144625,
          0.33057427,  0.22788048, -0.39905494,  0.45745587, -0.1793

NOTE: When we run block.eval(input) input should only be "one sample" (not a batch). However at training time we will feed data in (mini-) batches and fold will handle the variable input types and variable input sequence lenght between samples - _data dependent dynamic graph_ - and will also try to group operations together to actuall carry out computations "batch wise" - _dynamic batching_
